In [1]:
from pathlib import Path

# import numpy as np
import geopandas as gpd
import pandas as pd
import tomllib
from utils import read_dbf_and_groupby_sum, read_transit_assignments, time_periods

In [2]:
with open("transit.toml", "rb") as f:
    config = tomllib.load(f)

transit_line_rename_filepath = (
    Path(config["directories"]["resources"])
    / config["transit"]["line_rename_filename"]
)
model_run_dir = config["directories"]["model_run"]
model_MUNI_Line = config["muni"]["model_MUNI_Line"]
observed_MUNI_Line = Path(config["transit"]["observed_MUNI_Line"])
muni_name_match = Path(config["transit"]["muni_name_match"])
transit_input_dir = Path(config["directories"]["transit_input_dir"])
# output_dir = model_run_dir / "validation_workbook" / "output"
# output_transit_dir = output_dir / "transit"
# output_transit_dir.mkdir(parents=True, exist_ok=True)
combined_gdf = read_transit_assignments(model_run_dir, time_periods)

Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEA.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAAM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAMD.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAPM.DBF


In [154]:
Base_model_dir = Path(config["directories"]["Base_model_dir"])
FREEFLOW_SHP = Base_model_dir / config["transit"]["FREEFLOW_SHP"]

In [155]:
freeflow = gpd.read_file(FREEFLOW_SHP)
freeflow

In [156]:
freeflow

,A,B,TOLL,USE,CAP,AT,FT,STREETNAME,TYPE,MTYPE,...,ONEWAY,PROJ,DTA_EDIT_F,TOLLTIME,PHASE,A_B,ACTION,AB,GisLen,geometry
0,1,20368,0,1,2000,3,6,None,None,SF,...,0,None,0,0.0,0,None,None,1 20368,74.14878,"LINESTRING (6008120.306 2086464.601, 6008172.0..."
1,1,20373,0,1,2000,3,6,None,None,SF,...,0,None,0,0.0,0,None,None,1 20373,390.58781,"LINESTRING (6008120.306 2086464.601, 6007980.8..."
2,1,20375,0,1,2000,3,6,None,None,SF,...,0,None,0,0.0,0,None,None,1 20375,529.08099,"LINESTRING (6008120.306 2086464.601, 6008535.0..."
3,1,20408,0,1,2000,3,6,None,None,SF,...,0,None,0,0.0,0,None,None,1 20408,208.45584,"LINESTRING (6008120.306 2086464.601, 6007959.5..."
4,1,33349,0,1,2000,3,6,None,None,SF,...,0,None,0,0.0,0,None,None,1 33349,253.32850,"LINESTRING (6008120.306 2086464.601, 6007880.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62023,103998,102738,0,1,950,4,7,None,None,MTC,...,0,None,0,0.0,0,None,None,103998 102738,2160.39696,"LINESTRING (6145866.220 2089806.079, 6147362.1..."
62024,103998,102741,0,1,1400,4,5,None,None,MTC,...,0,None,0,0.0,0,None,None,103998 102741,1301.15446,"LINESTRING (6145866.220 2089806.079, 6144768.7..."
62025,103999,4026,0,1,950,4,7,None,None,MTC,...,0,None,0,0.0,0,None,None,103999 4026,1326.23564,"LINESTRING (6068042.745 2125565.525, 6069067.9..."
62026,103999,4030,0,1,1400,4,5,None,None,MTC,...,0,None,0,0.0,0,None,None,103999 4030,1525.92236,"LINESTRING (6068042.745 2125565.525, 6069366.5..."


In [130]:
station_node_match = Path(config["transit"]["station_node_match"])
model_BART = Path(config["bart"]["model_BART"])
observed_BART = Path(config["transit"]["observed_BART"])
output_dir = model_run_dir / "validation_workbook" / "output"
output_transit_dir = output_dir / "transit"

In [4]:
def map_name_to_direction(name):
    if name.endswith("I"):
        return "IB"
    elif name.endswith("O"):
        return "OB"
    else:
        return None  # Return None for other cases

In [27]:
combined_gdf

,A,B,TIME,MODE,FREQ,PLOT,COLOR,STOP_A,STOP_B,DIST,...,AB_XITA,AB_BRDB,AB_XITB,BA_VOL,BA_BRDA,BA_XITA,BA_BRDB,BA_XITB,geometry,TOD
0,1,11077,6041,3,0.0,0,0,0,0,3546,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,None,EA
1,1,11804,8183,3,0.0,0,0,0,0,5038,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,None,EA
2,1,11805,8186,3,0.0,0,0,0,0,5071,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,None,EA
3,1,12291,6119,3,0.0,0,0,0,0,3577,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,None,EA
4,1,12490,6031,3,0.0,0,0,0,0,3540,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,None,EA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150940,103998,4175,297,19,30.0,0,0,1,1,28,...,0.0,12.94,7.69,0.0,0.0,0.0,0.0,0.0,None,EV
1150941,103999,4026,500,5,0.0,0,0,0,0,25,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,None,EV
1150942,103999,4026,143,18,22.0,0,0,1,0,25,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,None,EV
1150943,103999,4026,143,22,20.0,0,0,1,1,25,...,0.0,5.84,9.84,0.0,0.0,0.0,0.0,0.0,None,EV


In [29]:
rename = pd.read_csv(transit_line_rename_filepath)
obs_model_name_match = pd.read_csv(transit_input_dir / muni_name_match)
obs_model_name_match = obs_model_name_match[["obs_line", "model_fullname"]]
obs_model_name_match = obs_model_name_match.rename(
    columns={"model_fullname": "FULLNAME"}
)
obs_model_name_match = obs_model_name_match.drop_duplicates()

MUNI = read_dbf_and_groupby_sum(
    combined_gdf, "SF MUNI", ["FULLNAME", "MODE","NAME", "TOD"], "AB_BRDA"
)
# MUNI["Direction"] = MUNI["NAME"].apply(map_name_to_direction)

# MUNI = MUNI.sort_values(by="FULLNAME").reset_index(drop=True)
# # Merge df1 with df2 based on the Name/NAME column
# df_merged = pd.merge(MUNI, rename[["NAME", "New"]], on="NAME", how="left")

# # # Update the FULLNAME in df1 with the New value from df2 if a match is found
# df_merged["FULLNAME"] = df_merged["New"].combine_first(df_merged["FULLNAME"])
# df_merged = df_merged.drop(columns=["New"])
# MUNI = MUNI.rename(columns={"NAME": "Name", "AB_BRDA": "Ridership"})

# MUNI_full = pd.merge(df_merged, obs_model_name_match, on="FULLNAME", how="left")
# MUNI_full = MUNI_full.rename(
#     columns={"NAME": "Name", "AB_BRDA": "Ridership", "obs_line": "Line"}
# )

#     # Apply the transformation function to the 'Line' column
obs_MUNI_line = pd.read_csv(transit_input_dir / observed_MUNI_Line)
# mode = obs_MUNI_line[["Line", "Mode"]].drop_duplicates().reset_index(drop=True)
# mode_dict = mode.set_index("Line")["Mode"].to_dict()
# MUNI_full["Mode"] = MUNI_full["Line"].map(mode_dict)
# MUNI_full["Key_line_dir"] = MUNI_full["Line"].astype(str) + MUNI_full["Direction"]
# MUNI_full["Key_line_tod"] = (
#     MUNI_full["Line"].astype(str) + MUNI_full["TOD"] + MUNI_full["Direction"]
# )
# MUNI_full = MUNI_full[
#     [
#         "Line",
#         "Mode",
#         "Direction",
#         "TOD",
#         "Key_line_dir",
#         "Key_line_tod",
#         "Ridership",
#         "Name",
#     ]
# ]
# MUNI_full = MUNI_full.sort_values(by=["Line", "Direction", "TOD"]).reset_index(
#     drop=True
# )

MUNI

In [31]:
MUNI[["NAME", "MODE"]].drop_duplicates()

,NAME,MODE
0,MUN1I,11
5,MUN1O,11
10,MUN1SHORTI,11
12,MUN1SHORTO,11
14,MUN10I,11
...,...,...
529,MUNMO,15
534,MUNNI,15
539,MUNNO,15
544,MUNNXI,12


In [34]:
rapid = MUNI[MUNI['NAME'].str.contains('SHORT')]
rapid

,FULLNAME,MODE,NAME,TOD,AB_BRDA
10,1 - CALIFORNIA,11,MUN1SHORTI,AM,1453.62
11,1 - CALIFORNIA,11,MUN1SHORTI,PM,1441.54
12,1 - CALIFORNIA,11,MUN1SHORTO,AM,1228.90
13,1 - CALIFORNIA,11,MUN1SHORTO,PM,2037.70
52,14 - MISSION,11,MUN14SHORTI,AM,1858.54
53,14 - MISSION,11,MUN14SHORTO,PM,1646.60
102,2 - CLEMENT,11,MUN2SHORTI,AM,343.38
103,2 - CLEMENT,11,MUN2SHORTI,PM,295.91
104,2 - CLEMENT,11,MUN2SHORTO,AM,412.36
105,2 - CLEMENT,11,MUN2SHORTO,PM,516.44


In [55]:
MUNI[MUNI['NAME'].str.contains('94')]

,FULLNAME,MODE,NAME,TOD,AB_BRDA
468,94L - OWL,11,MUN94LI,EA,63.77
469,94L - OWL,11,MUN94LO,EA,23.92


In [35]:
express = MUNI[MUNI['NAME'].str.contains('X')]
express

,FULLNAME,MODE,NAME,TOD,AB_BRDA
62,14X - MISSION EXPRESS,12,MUN14XI,AM,1263.81
63,14X - MISSION EXPRESS,12,MUN14XO,PM,1120.21
92,1AX - CALIFORNIA 'A' EXPRESS,12,MUN1AXI,AM,297.28
93,1AX - CALIFORNIA 'A' EXPRESS,12,MUN1AXO,PM,75.03
94,1BX - CALIFORNIA 'B' EXPRESS,12,MUN1BXI,AM,827.75
95,1BX - CALIFORNIA 'B' EXPRESS,12,MUN1BXO,PM,81.14
200,30 - STOCKTON,12,MUN30XI,AM,2169.73
201,30 - STOCKTON,12,MUN30XO,PM,1244.98
212,31AX - BALBOA 'A' EXPRESS,12,MUN31AXI,AM,162.06
213,31AX - BALBOA 'A' EXPRESS,12,MUN31AXO,PM,251.87


In [36]:
#EXPRESS
MUNI[MUNI['MODE'] == 12]
# 14X, 1AX, 1BX, 31AX, 31Bx, 38AX, 38BX, 81X, 82X suspended
# 55, 30





,FULLNAME,MODE,NAME,TOD,AB_BRDA
62,14X - MISSION EXPRESS,12,MUN14XI,AM,1263.81
63,14X - MISSION EXPRESS,12,MUN14XO,PM,1120.21
92,1AX - CALIFORNIA 'A' EXPRESS,12,MUN1AXI,AM,297.28
93,1AX - CALIFORNIA 'A' EXPRESS,12,MUN1AXO,PM,75.03
94,1BX - CALIFORNIA 'B' EXPRESS,12,MUN1BXI,AM,827.75
95,1BX - CALIFORNIA 'B' EXPRESS,12,MUN1BXO,PM,81.14
200,30 - STOCKTON,12,MUN30XI,AM,2169.73
201,30 - STOCKTON,12,MUN30XO,PM,1244.98
212,31AX - BALBOA 'A' EXPRESS,12,MUN31AXI,AM,162.06
213,31AX - BALBOA 'A' EXPRESS,12,MUN31AXO,PM,251.87


In [37]:
MUNI[MUNI['MODE'] == 14]

,FULLNAME,MODE,NAME,TOD,AB_BRDA
384,59 - POWELL/MASON,14,MUN59I,AM,239.12
385,59 - POWELL/MASON,14,MUN59I,EV,1409.15
386,59 - POWELL/MASON,14,MUN59I,MD,1431.98
387,59 - POWELL/MASON,14,MUN59I,PM,781.89
388,59 - POWELL/MASON,14,MUN59O,AM,449.17
389,59 - POWELL/MASON,14,MUN59O,EV,1486.26
390,59 - POWELL/MASON,14,MUN59O,MD,1755.84
391,59 - POWELL/MASON,14,MUN59O,PM,834.48
402,60 - POWELL/HYDE,14,MUN60I,AM,805.65
403,60 - POWELL/HYDE,14,MUN60I,EV,1863.59


In [40]:
MUNI[MUNI['MODE'] == 15]["FULLNAME"].drop_duplicates()

476    E - EMBARCADERO
484         F - MARKET
494         J - CHURCH
504      K - INGLESIDE
514        L - TARAVAL
524     M - OCEAN VIEW
534          N - JUDAH
Name: FULLNAME, dtype: object

In [32]:
MUNI["MODE"].drop_duplicates()


0      11
62     12
384    14
476    15
Name: MODE, dtype: int64

In [ ]:
11	Local Muni	local bus
12	Express Muni	local bus
14	Muni Cable Car	LRT
15	LRT Muni	LRT

In [13]:
rename

,NAME,Trn_asgn_new,Old,New
0,MUN8XI,MUN8I,8X Bayshore Express,8 Bayshore
1,MUN8XO,MUN8O,8X Bayshore Express,8 Bayshore
2,MUN16XI,MUN7XI,16X Noriega Express,7X Noreiga Express
3,MUN16XO,MUN7XO,16X Noriega Express,7X Noreiga Express
4,MUN17I,MUN57I,17 Parkmerced,18 Parkmerced
5,MUN17O,MUN57O,17 Parkmerced,18 Parkmerced
6,MUN71I,MUN7I,71 Haight-Noriega,7 Haight-Noriega
7,MUN71O,MUN7O,71 Haight-Noriega,7 Haight-Noriega
8,MUN108I,MUN25I,108 Treasure Island,25 Treasure Island
9,MUN108O,MUN25O,108 Treasure Island,25 Treasure Island


In [26]:
df_merged[df_merged["NAME"] == "MUN8XI"]

,FULLNAME,NAME,TOD,AB_BRDA,Direction
448,8 Bayshore,MUN8XI,EA,494.72,IB
449,8 Bayshore,MUN8XI,EV,3334.85,IB
450,8 Bayshore,MUN8XI,MD,4417.54,IB
451,8 Bayshore,MUN8XI,PM,2203.40,IB


In [10]:
obs_model_name_match

,obs_line,FULLNAME
0,1,1 - CALIFORNIA
2,2,2 - CLEMENT
4,3,3 - JACKSON
5,5,5 - FULTON
7,6,6 - PARNASSUS
...,...,...
91,F-Market & Wharves,F - MARKET
92,NaN,59 - POWELL/MASON
93,NaN,60 - POWELL/HYDE
94,NaN,61 - CALIFORNIA


In [16]:
print(list(MUNI['NAME'].drop_duplicates()))

['MUN1I', 'MUN1O', 'MUN1SHORTI', 'MUN1SHORTO', 'MUN10I', 'MUN10O', 'MUN108I', 'MUN108O', 'MUN12I', 'MUN12O', 'MUN14I', 'MUN14O', 'MUN14SHORTI', 'MUN14SHORTO', 'MUN14LI', 'MUN14LO', 'MUN14XI', 'MUN14XO', 'MUN17I', 'MUN17O', 'MUN18I', 'MUN18O', 'MUN19I', 'MUN19O', 'MUN1AXI', 'MUN1AXO', 'MUN1BXI', 'MUN1BXO', 'MUN2I', 'MUN2O', 'MUN2SHORTI', 'MUN2SHORTO', 'MUN21I', 'MUN21O', 'MUN22I', 'MUN22O', 'MUN23I', 'MUN23O', 'MUN24EAI', 'MUN24EAO', 'MUN24I', 'MUN24O', 'MUN27I', 'MUN27O', 'MUN28I', 'MUN28O', 'MUN28LI', 'MUN28LO', 'MUN29I', 'MUN29O', 'MUN3I', 'MUN3O', 'MUN30I', 'MUN30O', 'MUN30XI', 'MUN30XO', 'MUN31I', 'MUN31O', 'MUN31AXI', 'MUN31AXO', 'MUN31BXI', 'MUN31BXO', 'MUN33I', 'MUN33O', 'MUN35I', 'MUN35O', 'MUN36I', 'MUN36O', 'MUN37I', 'MUN37O', 'MUN38FTMI', 'MUN38FTMO', 'MUN38PTLI', 'MUN38PTLO', 'MUN38_33RDI', 'MUN38_33RDO', 'MUN38AXI', 'MUN38AXO', 'MUN38BXI', 'MUN38BXO', 'MUN38LI', 'MUN38LO', 'MUN39I', 'MUN39O', 'MUN41I', 'MUN41O', 'MUN43I', 'MUN43O', 'MUN44I', 'MUN44O', 'MUN45I', 'MUN45O', '

In [21]:
print(list(df_merged['NAME'].drop_duplicates()))

['MUN1I', 'MUN1SHORTO', 'MUN1SHORTI', 'MUN1O', 'MUN10O', 'MUN10I', 'MUN108O', 'MUN108I', 'MUN12O', 'MUN12I', 'MUN14SHORTO', 'MUN14O', 'MUN14SHORTI', 'MUN14I', 'MUN14LO', 'MUN14LI', 'MUN14XI', 'MUN14XO', 'MUN17O', 'MUN17I', 'MUN18O', 'MUN18I', 'MUN19O', 'MUN19I', 'MUN1AXI', 'MUN1AXO', 'MUN1BXI', 'MUN1BXO', 'MUN2SHORTI', 'MUN2SHORTO', 'MUN2O', 'MUN2I', 'MUN21O', 'MUN21I', 'MUN22O', 'MUN22I', 'MUN23O', 'MUN23I', 'MUN24O', 'MUN24I', 'MUN24EAO', 'MUN24EAI', 'MUN27O', 'MUN27I', 'MUN28O', 'MUN28I', 'MUN28LI', 'MUN28LO', 'MUN29O', 'MUN29I', 'MUN3O', 'MUN3I', 'MUN30O', 'MUN30XI', 'MUN30XO', 'MUN30I', 'MUN31O', 'MUN31I', 'MUN31AXI', 'MUN31AXO', 'MUN31BXI', 'MUN31BXO', 'MUN33O', 'MUN33I', 'MUN35O', 'MUN35I', 'MUN36I', 'MUN36O', 'MUN37O', 'MUN37I', 'MUN38_33RDO', 'MUN38_33RDI', 'MUN38PTLO', 'MUN38FTMO', 'MUN38PTLI', 'MUN38FTMI', 'MUN38AXI', 'MUN38AXO', 'MUN38BXI', 'MUN38BXO', 'MUN38LO', 'MUN38LI', 'MUN39I', 'MUN39O', 'MUN41I', 'MUN41O', 'MUN43O', 'MUN43I', 'MUN44O', 'MUN44I', 'MUN45I', 'MUN45O', '

In [18]:
obs_MUNI_line

,Line,Mode,Direction,TOD,Ridership
0,1,Local Bus,IB,AM,1786
1,1,Local Bus,IB,EA,108
2,1,Local Bus,IB,EV,609
3,1,Local Bus,IB,MD,4010
4,1,Local Bus,IB,PM,1623
...,...,...,...,...,...
531,F-Market & Wharves,Streetcar,IB,PM,545
532,F-Market & Wharves,Streetcar,OB,AM,116
533,F-Market & Wharves,Streetcar,OB,EV,478
534,F-Market & Wharves,Streetcar,OB,MD,1405


In [56]:
model_run_dir = Path(config["directories"]["model_run"])
transit_input_dir = Path(config["directories"]["transit_input_dir"])
MUNI_output_dir = Path(config["directories"]["MUNI_output_dir"])
markdown_output_dir = Path(config["directories"]["markdown_output_dir"])
Screenline_output_dir = Path(config["directories"]["Screenline_output_dir"])
BART_output_dir = Path(config["directories"]["BART_output_dir"])
observed_BART = Path(config["transit"]["observed_BART"])
observed_BART_county = Path(config["transit"]["observed_BART_county"])
observed_BART_Screenline = Path(config["transit"]["observed_BART_Screenline"])
observed_MUNI_Line = Path(config["transit"]["observed_MUNI_Line"])
observed_Screenline = Path(config["transit"]["observed_Screenline"])
observed_NTD = Path(config["transit"]["observed_NTD"])
model_BART = Path(config["bart"]["model_BART"])
model_BART_county = Path(config["bart"]["model_BART_county"])
model_BART_Screenline = Path(config["bart"]["model_BART_Screenline"])
model_MUNI_Line = Path(config["muni"]["model_MUNI_Line"])
model_Screenline = Path(config["screenline"]["model_Screenline"])
transbay_BART_IB_csv = Path(config["screenline"]["transbay_BART_IB_csv"])
transbay_BART_OB_csv = Path(config["screenline"]["transbay_BART_OB_csv"])
Countyline_BART_IB_csv = Path(config["screenline"]["Countyline_BART_IB_csv"])
Countyline_BART_OB_csv = Path(config["screenline"]["Countyline_BART_OB_csv"])
Intra_SF_BART_IB_csv = Path(config["screenline"]["Intra_SF_BART_IB_csv"])
Intra_SF_BART_OB_csv = Path(config["screenline"]["Intra_SF_BART_OB_csv"])
transbay_AC_IB_csv = Path(config["screenline"]["transbay_AC_IB_csv"])
transbay_AC_OB_csv = Path(config["screenline"]["transbay_AC_OB_csv"])
transbay_overall_IB_csv = Path(config["screenline"]["transbay_overall_IB_csv"])
transbay_overall_OB_csv = Path(config["screenline"]["transbay_overall_OB_csv"])

In [61]:
def convert_to_integer(value):
    try:
        return int(value)
    except ValueError:
        return value

# Valdiation for BART
def append_group_total(df, stations, group_name):
    # Filter the DataFrame for the specified stations
    df_group = df[df["Station"].isin(stations)]

    # Calculate the sum for the filtered rows
    group_total = pd.Series(df_group[["Observed", "Modeled"]].sum(), name=group_name)

    # Append the total row to the DataFrame with the group name
    df = pd.concat([df, group_total.to_frame().T], ignore_index=True)
    df.at[df.index[-1], "Station"] = group_name
    return df

def process_bart_data(
    obs_bart_line,
    model_bart_line,
    direction_filter,
    tod_filter,
    groupby_column,
    sum_column,
):
    """
    Process BART data from Excel and CSV files with flexible grouping and summing.

    Parameters:
    obs_bart_line (str): Path to the obs_bart CSV file.
    model_bart_line (str): Path to the model_bart CSV file.
    direction_filter (str): Direction to filter the data by.
    tod_filter (str): Time of Day
    groupby_column (str): Column name to group by.
    sum_column (str): Column name to sum.

    Returns:
    pd.DataFrame: The processed and formatted DataFrame.
    """

    # Apply the custom function to convert values
    model_bart_line[groupby_column] = model_bart_line[groupby_column].apply(
        convert_to_integer
    )

    # Determine filter conditions
    obs_condition = pd.Series([True] * len(obs_bart_line))
    model_condition = pd.Series([True] * len(model_bart_line))

    if direction_filter is not None:
        obs_condition &= obs_bart_line["Direction"] == direction_filter
        model_condition &= model_bart_line["Direction"] == direction_filter

    if tod_filter is not None:
        obs_condition &= obs_bart_line["TOD"] == tod_filter
        model_condition &= model_bart_line["TOD"] == tod_filter

    # Processing observed data
    MUNI_IB_obs = (
        obs_bart_line[obs_condition]
        .groupby(groupby_column)[sum_column]
        .sum()
        .reset_index()
    )
    MUNI_IB_obs.rename(
        columns={groupby_column: "Station", sum_column: "Observed"}, inplace=True
    )

    # Processing modeled data
    MUNI_IB_model = (
        model_bart_line[model_condition]
        .groupby(groupby_column)[sum_column]
        .sum()
        .reset_index()
    )
    MUNI_IB_model.rename(
        columns={groupby_column: "Station", sum_column: "Modeled"}, inplace=True
    )

    # Merging observed and modeled data
    MUNI_IB = pd.merge(MUNI_IB_obs, MUNI_IB_model, on="Station", how="left")
    all = [
        "EMBR",
        "MONT",
        "POWL",
        "CIVC",
        "16TH",
        "24TH",
        "GLEN",
        "BALB",
        "12TH",
        "19TH",
        "DALY",
    ]
    MUNI_IB = MUNI_IB[MUNI_IB["Station"].isin(all)]

    # Calculating total row and appending it
    # Define your station groups
    station_core_SF = ["EMBR", "MONT", "POWL", "CIVC"]
    station_outer_SF = ["16TH", "24TH", "GLEN", "BALB"]
    station_oakland = ["12TH", "19TH"]

    # Use the function to append totals for each group
    MUNI_IB = append_group_total(MUNI_IB, station_core_SF, "Core SF")
    MUNI_IB = append_group_total(MUNI_IB, station_outer_SF, "Outer SF")
    MUNI_IB = append_group_total(MUNI_IB, station_oakland, "Oakland Core")
    MUNI_IB = append_group_total(MUNI_IB, all, "All Listed Stations")

    # Calculating 'Diff' and 'Percentage Diff'
    MUNI_IB["Diff"] = MUNI_IB["Modeled"] - MUNI_IB["Observed"]
    MUNI_IB["Percentage Diff"] = MUNI_IB["Diff"] * 100 / MUNI_IB["Observed"]

    # Formatting the DataFrame
    numeric_cols = ["Observed", "Modeled", "Diff"]
    MUNI_IB = format_dataframe(
        MUNI_IB,
        numeric_columns=numeric_cols,
        percentage_columns=["Percentage Diff"],
    )

    return MUNI_IB

def process_data(
    obs_MUNI_line,
    model_MUNI_line,
    filters,
    groupby_column,
    sum_column,
    rename_column,
    join_method,
):
    """
    Process MUNI data with flexible filtering, grouping, and summing.

    Parameters:
    obs_MUNI_line (DataFrame): The observed MUNI line data.
    model_MUNI_line (DataFrame): The modeled MUNI line data.
    filters (list of tuples): List of filter conditions, where each tuple contains (column_name, value).
    groupby_column (str): Column name to group by.
    sum_column (str): Column name to sum.
    rename_column (str): New name to rename the groupby column to in the output.

    Returns:
    DataFrame: The processed and formatted DataFrame.
    """

    # Apply the custom function to convert values
    model_MUNI_line[groupby_column] = model_MUNI_line[groupby_column].apply(
        convert_to_integer
    )
    obs_MUNI_line[groupby_column] = obs_MUNI_line[groupby_column].apply(
        convert_to_integer
    )

    # Apply filters
    if filters is not None:
        for filter_col, filter_val in filters:
            obs_MUNI_line = obs_MUNI_line[obs_MUNI_line[filter_col] == filter_val]
            model_MUNI_line = model_MUNI_line[model_MUNI_line[filter_col] == filter_val]

    # Processing observed data
    MUNI_IB_obs = obs_MUNI_line.groupby(groupby_column)[sum_column].sum().reset_index()
    MUNI_IB_obs.rename(
        columns={groupby_column: rename_column, sum_column: "Observed"}, inplace=True
    )

    # Processing modeled data
    MUNI_IB_model = (
        model_MUNI_line.groupby(groupby_column)[sum_column].sum().reset_index()
    )
    MUNI_IB_model.rename(
        columns={groupby_column: rename_column, sum_column: "Modeled"}, inplace=True
    )

    # Merging observed and modeled data
    MUNI_IB = pd.merge(MUNI_IB_obs, MUNI_IB_model, on=rename_column, how=join_method)

    # Calculating total row and appending it
    total_row = pd.Series(MUNI_IB[["Observed", "Modeled"]].sum(), name="Total")
    MUNI_IB = pd.concat([MUNI_IB, total_row.to_frame().T], ignore_index=True)

    # Calculating 'Diff' and 'Percentage Diff'
    MUNI_IB["Diff"] = MUNI_IB["Modeled"] - MUNI_IB["Observed"]
    MUNI_IB["Percentage Diff"] = MUNI_IB["Diff"] * 100 / MUNI_IB["Observed"]

    # Assign 'Total' label to the last row in the renamed column
    MUNI_IB.at[MUNI_IB.index[-1], rename_column] = "Total"

    # Formatting the DataFrame
    numeric_cols = ["Observed", "Modeled", "Diff"]
    MUNI_IB = format_dataframe(
        MUNI_IB, numeric_columns=numeric_cols, percentage_columns=["Percentage Diff"]
    )

    return MUNI_IB



In [58]:
from utils import dataframe_to_markdown, format_dataframe

In [85]:
output_dir = model_run_dir / "validation_workbook" / "output"
output_transit_dir = output_dir / "transit"
obs_Screenline = pd.read_csv(transit_input_dir / observed_Screenline)
model_Screenline_df = pd.read_csv(output_transit_dir / model_Screenline)
transbay_overall_IB = process_data(
    obs_Screenline,
    model_Screenline_df,
    [("Screenline", "Transbay"), ("Direction", "IB")],
    "TOD",
    "Ridership",
    "TOD",
    "left",
)

In [73]:
model_Screenline_df

,Screenline,Direction,TOD,Key,Ridership,Operator,Mode
0,Transbay,IB,AM,TransbayBARTAMIB,56079,BART,BART
1,Transbay,IB,EA,TransbayBARTEAIB,8072,BART,BART
2,Transbay,IB,EV,TransbayBARTEVIB,2513,BART,BART
3,Transbay,IB,MD,TransbayBARTMDIB,25334,BART,BART
4,Transbay,IB,PM,TransbayBARTPMIB,6487,BART,BART
...,...,...,...,...,...,...,...
69,Transbay,OB,AM,TransbayAC TransitAMOB,159,AC Transit,Premium
70,Transbay,OB,EA,TransbayAC TransitEAOB,46,AC Transit,Premium
71,Transbay,OB,EV,TransbayAC TransitEVOB,2280,AC Transit,Premium
72,Transbay,OB,MD,TransbayAC TransitMDOB,574,AC Transit,Premium


In [90]:
model_Screenline_df.groupby(['Direction', 'TOD']).sum('Ridership').reset_index()

,Direction,TOD,Ridership
0,IB,AM,99563.29
1,IB,EA,12556.66
2,IB,EV,9405.53
3,IB,MD,55424.10
4,IB,PM,23777.59
5,OB,AM,20360.98
6,OB,EA,2380.40
7,OB,EV,33994.24
8,OB,MD,53504.73
9,OB,PM,90232.69


In [92]:
screenline_overall_ib = process_data(
    obs_Screenline,
    model_Screenline_df,
    [("Direction", "IB")],
    "TOD",
    "Ridership",
    "TOD",
    "left",
)
screenline_overall

,TOD,Observed,Modeled,Diff,Percentage Diff
0,AM,"67,259","99,563","32,304",48%
1,EA,"3,755","12,557","8,802",234%
2,EV,"19,093","9,406","-9,687",-51%
3,MD,"65,891","55,424","-10,467",-16%
4,PM,"27,578","23,778","-3,800",-14%
5,Total,"183,576","200,727","17,151",9%


In [89]:
obs_Screenline.groupby(['Direction', 'TOD']).sum('Ridership').reset_index()

,Direction,TOD,Ridership
0,IB,AM,67259
1,IB,EA,3755
2,IB,EV,19093
3,IB,MD,65891
4,IB,PM,27578
5,OB,AM,15615
6,OB,EA,576
7,OB,EV,46294
8,OB,MD,42381
9,OB,PM,73892


In [86]:
model_Screenline_df

,Screenline,Direction,TOD,Key,Ridership,Operator,Mode
0,Transbay,IB,AM,TransbayBARTAMIB,56079.90,BART,BART
1,Transbay,IB,EA,TransbayBARTEAIB,8072.47,BART,BART
2,Transbay,IB,EV,TransbayBARTEVIB,2513.16,BART,BART
3,Transbay,IB,MD,TransbayBARTMDIB,25334.91,BART,BART
4,Transbay,IB,PM,TransbayBARTPMIB,6487.78,BART,BART
...,...,...,...,...,...,...,...
69,Transbay,OB,AM,TransbayAC TransitAMOB,159.29,AC Transit,Premium
70,Transbay,OB,EA,TransbayAC TransitEAOB,46.53,AC Transit,Premium
71,Transbay,OB,EV,TransbayAC TransitEVOB,2280.90,AC Transit,Premium
72,Transbay,OB,MD,TransbayAC TransitMDOB,574.19,AC Transit,Premium


In [87]:
obs_Screenline

,Screenline,Direction,TOD,Ridership,Operator,Mode,Key
0,Countyline,IB,AM,8873,BART,BART,CountylineBARTAMIB
1,Countyline,IB,EA,439,BART,BART,CountylineBARTEAIB
2,Countyline,IB,EV,6098,BART,BART,CountylineBARTEVIB
3,Countyline,IB,MD,12278,BART,BART,CountylineBARTMDIB
4,Countyline,IB,PM,7364,BART,BART,CountylineBARTPMIB
...,...,...,...,...,...,...,...
60,Transbay,OB,PM,453,AC Transit,Premium,TransbayAC TransitPMOB
61,Transbay,IB,AM,394,AC Transit,Premium,TransbayAC TransitAMIB
62,Transbay,IB,EV,666,AC Transit,Premium,TransbayAC TransitEVIB
63,Transbay,IB,MD,1389,AC Transit,Premium,TransbayAC TransitMDIB


In [84]:
obs_Screenline["TOD"] = obs_Screenline["TOD"].apply(
    convert_to_integer
)
obs_Screenline = obs_Screenline.groupby(["TOD", "Direction"])["Ridership"].sum().reset_index()
obs_Screenline.rename(
    columns={"TOD": "TOD", "Ridership": "Observed"}, inplace=True
)

KeyError: 'Column not found: Ridership'

In [83]:
obs_Screenline

,TOD,Direction,Observed
0,AM,IB,67259
1,AM,OB,15615
2,EA,IB,3755
3,EA,OB,576
4,EV,IB,19093
5,EV,OB,46294
6,MD,IB,65891
7,MD,OB,42381
8,PM,IB,27578
9,PM,OB,73892


In [78]:
model_Screenline_df

,Screenline,Direction,TOD,Key,Ridership,Operator,Mode
0,Transbay,IB,AM,TransbayBARTAMIB,56079,BART,BART
1,Transbay,IB,EA,TransbayBARTEAIB,8072,BART,BART
2,Transbay,IB,EV,TransbayBARTEVIB,2513,BART,BART
3,Transbay,IB,MD,TransbayBARTMDIB,25334,BART,BART
4,Transbay,IB,PM,TransbayBARTPMIB,6487,BART,BART
5,Transbay,OB,AM,TransbayBARTAMOB,4940,BART,BART
6,Transbay,OB,EA,TransbayBARTEAOB,663,BART,BART
7,Transbay,OB,EV,TransbayBARTEVOB,16131,BART,BART
8,Transbay,OB,MD,TransbayBARTMDOB,25352,BART,BART
9,Transbay,OB,PM,TransbayBARTPMOB,49445,BART,BART


In [77]:
obs_Screenline

,Screenline,Direction,TOD,Ridership,Operator,Mode,Key
10,Transbay,IB,AM,46176,BART,BART,TransbayBARTAMIB
11,Transbay,IB,EA,2717,BART,BART,TransbayBARTEAIB
12,Transbay,IB,EV,10257,BART,BART,TransbayBARTEVIB
13,Transbay,IB,MD,41645,BART,BART,TransbayBARTMDIB
14,Transbay,IB,PM,13446,BART,BART,TransbayBARTPMIB
15,Transbay,OB,AM,6607,BART,BART,TransbayBARTAMOB
16,Transbay,OB,EA,252,BART,BART,TransbayBARTEAOB
17,Transbay,OB,EV,33067,BART,BART,TransbayBARTEVOB
18,Transbay,OB,MD,22826,BART,BART,TransbayBARTMDOB
19,Transbay,OB,PM,49312,BART,BART,TransbayBARTPMOB


In [76]:
# Apply the custom function to convert values
model_Screenline_df["Ridership"] = model_Screenline_df["Ridership"].apply(
    convert_to_integer
)

obs_Screenline = obs_Screenline[obs_Screenline["Screenline"] == "Transbay"]
model_Screenline_df = model_Screenline_df[model_Screenline_df["Screenline"] == "Transbay"]

In [65]:
obs_Screenline

,Screenline,Direction,TOD,Ridership,Operator,Mode,Key
0,Countyline,IB,AM,8873,BART,BART,CountylineBARTAMIB
1,Countyline,IB,EA,439,BART,BART,CountylineBARTEAIB
2,Countyline,IB,EV,6098,BART,BART,CountylineBARTEVIB
3,Countyline,IB,MD,12278,BART,BART,CountylineBARTMDIB
4,Countyline,IB,PM,7364,BART,BART,CountylineBARTPMIB
...,...,...,...,...,...,...,...
60,Transbay,OB,PM,453,AC Transit,Premium,TransbayAC TransitPMOB
61,Transbay,IB,AM,394,AC Transit,Premium,TransbayAC TransitAMIB
62,Transbay,IB,EV,666,AC Transit,Premium,TransbayAC TransitEVIB
63,Transbay,IB,MD,1389,AC Transit,Premium,TransbayAC TransitMDIB


In [64]:
transbay_overall_IB

,TOD,Observed,Modeled,Diff,Percentage Diff
0,AM,"46,570","57,994","11,424",25%
1,EA,"2,717","9,246","6,529",240%
2,EV,"10,923","2,921","-8,002",-73%
3,MD,"43,034","25,848","-17,186",-40%
4,PM,"15,099","6,923","-8,176",-54%
5,Total,"118,343","102,932","-15,411",-13%


In [100]:
obs_MUNI_line_df = pd.read_csv(transit_input_dir / observed_MUNI_Line)
model_MUNI_line_df = pd.read_csv(output_transit_dir / model_MUNI_Line)

In [104]:
model_MUNI_line_df

,Line,Mode,Direction,TOD,Key_line_dir,Key_line_tod,Ridership,Name
0,001AX,NaN,IB,AM,001AXIB,001AXAMIB,297.28,MUN1AXI
1,001AX,NaN,OB,PM,001AXOB,001AXPMOB,75.03,MUN1AXO
2,001BX,NaN,IB,AM,001BXIB,001BXAMIB,827.75,MUN1BXI
3,001BX,NaN,OB,PM,001BXOB,001BXPMOB,81.14,MUN1BXO
4,005R,Express Bus,IB,AM,005RIB,005RAMIB,4371.94,MUN5LI
...,...,...,...,...,...,...,...,...
541,NaN,NaN,OB,PM,nanOB,nanPMOB,261.83,MUNNXO
542,NaN,NaN,NaN,AM,NaN,NaN,474.19,MUN61
543,NaN,NaN,NaN,EV,NaN,NaN,610.97,MUN61
544,NaN,NaN,NaN,MD,NaN,NaN,754.65,MUN61


In [101]:
obs_MUNI_line_df 

,Line,Mode,Direction,TOD,Ridership
0,1,Local Bus,IB,AM,1786
1,1,Local Bus,IB,EA,108
2,1,Local Bus,IB,EV,609
3,1,Local Bus,IB,MD,4010
4,1,Local Bus,IB,PM,1623
...,...,...,...,...,...
531,F-Market & Wharves,Streetcar,IB,PM,545
532,F-Market & Wharves,Streetcar,OB,AM,116
533,F-Market & Wharves,Streetcar,OB,EV,478
534,F-Market & Wharves,Streetcar,OB,MD,1405


In [102]:
MUNI_mode_df = process_data(
    obs_MUNI_line_df, model_MUNI_line_df, None, "Mode", "Ridership", "Mode", "left"
)

In [103]:
MUNI_mode_df

,Mode,Observed,Modeled,Diff,Percentage Diff
0,Express Bus,"7,994","106,480","98,486",1232%
1,Local Bus,"317,364","349,360","31,996",10%
2,Rail,"47,899","116,077","68,178",142%
3,Rapid,"64,395",-,-,-
4,Streetcar,"5,386","27,405","22,019",409%
5,Total,"443,038","599,321","156,283",35%


In [107]:
with open('X:\Projects\CHAMP7\Run25w_Market\muni.link', 'r') as file:
    for line in range(10):  # Read the first 5 lines
        print(file.readline())



;######################### From: Q:\Model Projects\DTX\s0_2019_Base\RunRecord\build\scratch\Wrangler_tmp_2021Feb08.045030\seed\Muni_2020_01\muni.link



;***************************************

; THESE ARE ADDITIONAL LINKS FOR MUNI ROUTES

LINK nodes=25984,26282, dist=100, modes=15, oneway=no, time=2.79 ; Sunset Tunnel

LINK nodes=25884,13405, dist=078, modes=15, oneway=no, time=4.33 ; Church/Duboce Portal

LINK nodes=25712,25708, dist=022, modes=15, oneway=no, time=1.42 ; J Right-of-way - Mission Dolores Park

LINK nodes=25708,24835, dist=008, modes=15, oneway=no, time=0.65 ; J Right-of-way - Liberty Hill

LINK nodes=24835,25661, dist=006, modes=15, oneway=no, time=0.43 ; J Right-of-way - Liberty Hill



In [115]:
import pandas as pd
import re

data = []
# Open and read the .link file
with open('X:\Projects\CHAMP7\Run25w_Market\muni.link', 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith("LINK"):  # Process only LINK lines
            # Use regular expression to extract fields
            match = re.search(r'nodes=(\d+),(\d+), dist=(\d+\.\d+|\d+), modes=(\d+), oneway=(\w+), time=(\d+\.\d+|\d+)\s*;\s*(.*)', line)
            if match:
                node1, node2, dist, modes, oneway, time, description = match.groups()
                # Append parsed data as a dictionary
                data.append({
                    'Node1': int(node1),
                    'Node2': int(node2),
                    'Distance': float(dist),
                    'Modes': int(modes),
                    'Oneway': oneway,
                    'Time': float(time),
                    'Description': description
                })

# Convert list of dictionaries into a DataFrame
df = pd.DataFrame(data)

df



,Node1,Node2,Distance,Modes,Oneway,Time,Description
0,25984,26282,100.0,15,no,2.79,Sunset Tunnel
1,25884,13405,78.0,15,no,4.33,Church/Duboce Portal
2,25712,25708,22.0,15,no,1.42,J Right-of-way - Mission Dolores Park
3,25708,24835,8.0,15,no,0.65,J Right-of-way - Liberty Hill
4,24835,25661,6.0,15,no,0.43,J Right-of-way - Liberty Hill
...,...,...,...,...,...,...,...
69,54001,54002,47.0,15,no,2.95,Folsom to Brannan
70,54002,54003,36.0,15,no,1.89,Brannan to 2nd
71,54003,54004,35.0,15,no,3.61,2nd to 4th (offstreet)
72,54003,23798,30.0,15,no,3.56,2nd to 4th onstreet (to allow traffic interact...


In [110]:
MUNI_map = read_dbf_and_groupby_sum(
    combined_gdf, "SF MUNI", ["FULLNAME", "NAME", "AB", "SEQ"], "AB_BRDA"
)

In [113]:
combined_gdf[combined_gdf["A"] == '25984']

,A,B,TIME,MODE,FREQ,PLOT,COLOR,STOP_A,STOP_B,DIST,...,AB_XITA,AB_BRDB,AB_XITB,BA_VOL,BA_BRDA,BA_XITA,BA_BRDB,BA_XITB,geometry,TOD


In [111]:
MUNI_map

,FULLNAME,NAME,AB,SEQ,AB_BRDA
0,1 - CALIFORNIA,MUN1I,24693 30050,91,22.41
1,1 - CALIFORNIA,MUN1I,24694 24693,90,0.00
2,1 - CALIFORNIA,MUN1I,24703 24694,89,117.79
3,1 - CALIFORNIA,MUN1I,24754 24703,88,0.00
4,1 - CALIFORNIA,MUN1I,24756 24754,87,324.56
...,...,...,...,...,...
13929,NXO,MUNNXO,27946 27947,89,0.00
13930,NXO,MUNNXO,27947 27950,90,10.41
13931,NXO,MUNNXO,27950 27951,91,0.00
13932,NXO,MUNNXO,28106 27441,57,0.00
